# CUDA JupyterLab Setup Test (Container-Based Directory Creation)

This notebook tests the CUDA JupyterLab environment and creates the necessary workspace and cache directories from within the container. This approach avoids permission issues with external storage mounting.

In [ ]:
# Test basic imports
import torch
import transformers
import sys
import os
from pathlib import Path

print("✓ All imports successful!")

In [ ]:
print("=== Setting up workspace and cache directories ===")

# Create workspace directory if it doesn't exist
workspace_path = Path.home() / 'workspace'
workspace_path.mkdir(exist_ok=True)
print(f"✓ Workspace directory created/verified: {workspace_path}")

# Create cache directory if it doesn't exist
cache_path = Path.home() / 'cache'
cache_path.mkdir(exist_ok=True)
print(f"✓ Cache directory created/verified: {cache_path}")

# Create HuggingFace cache subdirectory
hf_cache_path = cache_path / 'huggingface'
hf_cache_path.mkdir(exist_ok=True)
print(f"✓ HuggingFace cache directory created/verified: {hf_cache_path}")

# Set HuggingFace cache environment variable
os.environ['HF_HOME'] = str(hf_cache_path)
os.environ['TRANSFORMERS_CACHE'] = str(hf_cache_path)
print(f"✓ HuggingFace cache environment variables set:")
print(f"  HF_HOME: {os.environ['HF_HOME']}")
print(f"  TRANSFORMERS_CACHE: {os.environ['TRANSFORMERS_CACHE']}")

print("✓ Directory setup completed!")

In [ ]:
# Test CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
else:
    print("❌ CUDA not available!")

In [ ]:
# Test storage configuration (directories created from container)
print("=== Storage Configuration Test ===")
print(f"Current working directory: {os.getcwd()}")
print(f"Home directory: {Path.home()}")

# Check workspace (should exist after setup cell)
workspace_path = Path.home() / 'workspace'
print(f"Workspace path: {workspace_path}")
print(f"Workspace exists: {workspace_path.exists()}")
if workspace_path.exists():
    print(f"Workspace is writable: {os.access(workspace_path, os.W_OK)}")
else:
    print("❌ Workspace directory not found - run setup cell above first")

# Check cache directory (should exist after setup cell)
cache_path = Path.home() / 'cache'
print(f"Cache path: {cache_path}")
print(f"Cache exists: {cache_path.exists()}")
if cache_path.exists():
    print(f"Cache is writable: {os.access(cache_path, os.W_OK)}")
else:
    print("❌ Cache directory not found - run setup cell above first")

# HF cache specifically
hf_cache = cache_path / 'huggingface'
print(f"HF cache path: {hf_cache}")
print(f"HF cache exists: {hf_cache.exists()}")

# Check environment variables
print(f"HF_HOME environment variable: {os.environ.get('HF_HOME', 'Not set')}")
print(f"TRANSFORMERS_CACHE environment variable: {os.environ.get('TRANSFORMERS_CACHE', 'Not set')}")

print("✓ All directories created from within container successfully!")

In [ ]:
# Test Hugging Face transformers with local cache
from transformers import AutoTokenizer, AutoModel

print("Testing Hugging Face model download (cache on disk B)...")
model_name = "distilbert-base-uncased"

try:
    # This will download to the cache directory on disk B
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    # Test basic functionality
    test_text = "Hello, this is a test of the Hugging Face integration with disk B storage!"
    inputs = tokenizer(test_text, return_tensors="pt")
    
    if torch.cuda.is_available():
        model = model.cuda()
        inputs = {k: v.cuda() for k, v in inputs.items()}
        print("✓ Model moved to GPU")
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    print(f"✓ Model loaded and tested successfully!")
    print(f"✓ Output shape: {outputs.last_hidden_state.shape}")
    print(f"✓ Model cached on disk B for future use")
    
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test flash attention if available
try:
    import flash_attn
    print(f"✓ Flash Attention available: {flash_attn.__version__}")
except ImportError:
    print("❌ Flash Attention not available (this is normal for some setups)")

In [ ]:
# Test GPU memory and performance
if torch.cuda.is_available():
    print("=== GPU Memory Test ===")
    
    # Clear cache
    torch.cuda.empty_cache()
    
    # Check initial memory
    print(f"Initial GPU memory: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
    
    # Create a tensor
    x = torch.randn(1000, 1000, device='cuda')
    y = torch.randn(1000, 1000, device='cuda')
    
    print(f"After tensor creation: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
    
    # Perform operation
    z = torch.matmul(x, y)
    
    print(f"After matrix multiplication: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
    print(f"✓ GPU computation successful!")
    
    # Clean up
    del x, y, z
    torch.cuda.empty_cache()
    print(f"After cleanup: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
else:
    print("❌ GPU not available for memory test")

In [ ]:
# Summary
print("\n" + "="*50)
print("        SETUP TEST SUMMARY")
print("="*50)
print(f"✓ JupyterLab running with NVIDIA PyTorch container")
print(f"✓ Python {sys.version.split()[0]}")
print(f"✓ PyTorch {torch.__version__}")
print(f"✓ Transformers {transformers.__version__}")
print(f"✓ CUDA: {'Available' if torch.cuda.is_available() else 'Not Available'}")
print(f"✓ Workspace and cache directories created from container")
print(f"✓ HF cache environment variables configured")
print(f"✓ HF models cache on external storage (via container)")
print(f"✓ No external folder creation required!")
print("\n🎉 Environment ready for ML/DL development!")
print("="*50)